In [1]:
import os
from operator import index
import math
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
import warnings
from sklearn.model_selection import train_test_split

plt.rcParams['figure.figsize'] = (10, 4)
folder_path = "../Spectrogram_Npy/"
sample_rate = 22050
duration = 30  # mesaured in seconds
samples_per_track = sample_rate * duration

In [2]:
def read_dataset():
    data = pd.read_csv("../data.csv")
    data = data[data.meter.notnull()]
    return data

In [3]:
def extract_spectrogram_array(num_segments=10):
   
    # loop through all songs
    alldata = read_dataset()
    num_samples_per_segment = int(samples_per_track / num_segments)
    
    data = alldata.groupby(['meter'])
    
    for meter, group in data:
        if not os.path.exists(str(folder_path + meter)):
            os.makedirs(os.path.join(folder_path, str(meter)))
            
        for index, song in group.iterrows():
            
            path = os.path.relpath("../audio_files" + song.path)
            signal, sr = librosa.load(path, sr=sample_rate)
            
            for s in range(num_segments):
                start_sample = num_samples_per_segment * s
                finish_sample = start_sample + num_samples_per_segment
                
                spec = np.abs(librosa.stft(signal[start_sample:finish_sample]))
                
                spectrogram = librosa.amplitude_to_db(spec, ref=np.max)
                
                file_name = "spec"+ str(index) + '-'+str(s)+".npy"
                full_name = os.path.join(folder_path, str(meter), file_name)
                np.save(full_name, spectrogram)
        
extract_spectrogram_array()